# Attractivity vs Stability

## Setup the notebook
Running the first cell below, you will download on Google's machines all the libraries you need to run this notebook.
The first time you run this, it should take approximately two minutes.
But the good news is that you will have to do this only once every 12 hours!

In [ ]:
# install drake and the underactuated source repo (only if necessary) and set up the path
# colab will ask you to "reset all runtimes", say no to save yourself the reinstall
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

In [ ]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt

# pydrake imports
from pydrake.all import DiagramBuilder, Variable, SymbolicVectorSystem, LogOutput, Simulator

## Dynamics of the nonlinear system
Here we create a Drake dynamical system using the class `SymbolicVectorSystem`.
This requires the dynamics to be written in the form $\dot{\mathbf{x}} = f(\mathbf{x})$, where $\mathbf{x}$ is a vector of Drake symbolic variables.
`SymbolicVectorSystem` is just one of the many options we have in drake: when systems will get more complicated, writing the dynamics by hand can be rather tedious...

In [ ]:
# function that given the current state returns the state time derivative
def dynamics(x1, x2):
    r = np.sqrt(x1**2 + x2**2)
    return [
        x1*(1-r) - x2*(r-x1)/(2*r),
        x2*(1-r) + x1*(r-x1)/(2*r)
    ]

# state variables
x1 = Variable("x1")
x2 = Variable("x2")

# drake nonlinear system
system = SymbolicVectorSystem(
    state=[x1, x2],
    output=[x1, x2], # all the state variables are measured
    dynamics=dynamics(x1, x2)
)

## Drake diagram
We then construct a Drake diagram.
This is nothing more than a set of interconnected dynamical systems (similiar to the Simulink idea, if you ever used it).
Our diagram will be very simple: we just connect our dynamical system to a logger, which will measure and store the system state during the simulation (similar to the Simulink Scope block).

In [ ]:
# initialize builder of the diagram
builder = DiagramBuilder()

# add our dynamical system
# (note: builder.AddSystem() returns a pointer to the system passed as input,
# hence it is safe to assign the name "system" to its output)
system = builder.AddSystem(system)

# logger block to measure and store the state
# connected to the (first and only) output port of the dynamical system
logger = LogOutput(system.get_output_port(0), builder)

# finalize diagram
diagram = builder.Build()

## Simulation
We are ready to simulate our dynamical system.
To this end we just feed our diagram in a the Drake `Simulator` and `AdvanceTo` the desired time.

In [ ]:
# function that given the initial state
# and a simulation time returns the system trajectory
def simulate(x1, x2, sim_time):
    
    # clean the logger from old trajectories
    logger.reset()
    
    # set up the simulator
    simulator = Simulator(diagram)
    
    # set initial conditions
    # (for now, think of "context" as a synonym of state)
    context = simulator.get_mutable_context()
    context.SetContinuousState([x1, x2])
    
    # simulate from t=0 to t=sim_time
    simulator.AdvanceTo(sim_time)
    
    # return the output (here = state) trajectory
    return logger.data()

## Plotting helpers
Let us write a couple of functions that will make the plotting code a little cleaner.
We have two functions here: one to plot the phase portrait in the background, another to plot the system trajectory on top.

In [ ]:
# function that given a function which implements xdot = f(x)
# for a 2d system plots the phase portrait
def plot_phase_portrait(
    dynamics,
    d1=2., # x1 dimension of the side of the phase portrait
    d2=2., # x2 dimension of the side of the phase portrait
    n=100j, # samples per side, must be a complex number (weird python indexing trick!)
    **kwargs # other keyword arguments for the streamplot function
):

    # grid state space
    X2, X1 = np.mgrid[-d1:d1:n, -d2:d2:n] # careful: x2 before x1!
    
    # evaluate the dynamics on the grid
    X1d, X2d = dynamics(X1, X2)
    
    # color the streamlines according to the magnitude of xdot
    color = np.sqrt(X1d**2 + X2d**2)
    
    # phase portrait
    strm = plt.streamplot(X1, X2, X1d, X2d, color=color, **kwargs)
    
    # colorbar on the right that measures the magnitude of xdot
    plt.gcf().colorbar(strm.lines, label=r'$|\dot\mathbf{x}|$')
    
    # misc plot settings
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.xlim(-d1, d1)
    plt.ylim(-d2, d2)
    plt.gca().set_aspect('equal') # equal aspect ratio for x1 and x2
    
# function that given the initial state
# and a simulation time plots the system trajectory
def plot_trajectory(x1, x2, sim_time):
    
    # simulate the system from the given state
    traj = simulate(x1, x2, sim_time)
    
    # plot a blue dot for the initial conditions
    label = r'$\mathbf{x}(0)=[%.2f,%.2f]^T$'%(x1,x2)
    plt.scatter(x1, x2, s=50, c='b', zorder=3, label=label)
    
    # plot a red curve for the trajectory
    label = r'$\mathbf{x}(t)$'
    plt.plot(traj[0,:], traj[1,:], label=label, c='r')

## Play with the initial conditions
Now we can finally visualize the dynamics of our system.
To do this, in the next cell modify the two variables (currently set to arbitrary values):
- `initial_conditions`: state of the system a time zero,
- `sim_time`: duration of the simulation in seconds.

Then, run the last cell to see the result of the simulation.

In [ ]:
initial_conditions = [-1, -1] # modify here
sim_time = 1 # modify here

In [ ]:
# plot and legend
plt.figure(figsize=(10, 10))
plot_phase_portrait(dynamics, linewidth=1, density=2)
plot_trajectory(*initial_conditions, sim_time)
plt.legend(loc=1)